**Introducimos datos nuevos y realizamos pipeline para limpiar datos:**

Primero vamos a transformar el dato numerico de tiempo venta y tiempo recogida a fechas:

In [ ]:
df = pd.DataFrame(df)
fecha_base = pd.to_datetime("2022-09-01 07:00")
df["fecha_hora_venta"] = fecha_base + pd.to_timedelta(df["tiempo_venta"], unit="h")
df.head()

In [ ]:
df = pd.DataFrame(df)
fecha_base = pd.to_datetime("2022-09-01 07:00")
df["fecha_hora_recogida"] = fecha_base + pd.to_timedelta(df["tiempo_recogida"], unit="h")
df

In [ ]:
fecha_base = pd.to_datetime("2022-09-01 07:00")
df["fecha_hora_recogida"] = fecha_base + pd.to_timedelta(df["tiempo_recogida"], unit="h")
df["fecha_hora_recogida"] = df["fecha_hora_recogida"].dt.strftime("%Y-%m-%d %H:%M:%S")
df

Redondeamos los decimales a dos

In [ ]:
df['coste_inicial'] = df['coste_inicial'].round(2)
df['peso'] = df['peso'].round(2)
df['precio_venta'] = df['precio_venta'].round(2)

df.head()

Comprobamos que el peso no sea negativo

In [ ]:
negativos = df[df["peso"] < 0]

Como en las anteriores hemos decidido eliminar pesos negativos, aqui también lo hacemos.

In [ ]:
filas_peso_negativo = df[df["peso"] < 0]
print(filas_peso_negativo)

Comprobamos que el peso cuadra con la fruta indicada.
Tendriamos que hacer un pipeline por cada fruta ya que un melón y una manzana no tienen pesos, precios ni nada comparable.

In [ ]:
print("Cantidad de pesos mayores de 120 g:", len(df[df['peso'] > 120]))
print("Peso máximo:", df.loc[df['peso'] > 120, 'peso'].max())

Comprobamos que no haya costes negativos ni precios venta negativos.

In [ ]:
negativos_coste = df[df["coste_inicial"] < 0]
negativos_pv = df[df["precio_venta"] < 0]

Y ahora comprobamos que no haya precio de venta nulos, ya que estos son frutas no vendidas por mal estado.

In [ ]:
nulos_pv = df[df["precio_venta"].isnull()]
nulos_pv[nulos_pv["fecha_venta"].isnull()]

Comprobamos que no se haya vendido más de 100KG de una fruta a un solo proveeedor en un solo dia.

In [ ]:
venta_dia = df.groupby(['proveedor', 'fecha_venta'])['peso'].sum()

mayor_100kg = (venta_dia > 100000).any()

print(f"¿Algún proveedor recibe más de 100 kg en un solo día? {mayor_100kg}")

Comprobamos si hay duplicados, para después valorar cómo tratarlos.

In [ ]:
duplicated_t_ids = df.loc[
    df.duplicated(subset='t_id', keep=False),
    't_id'
]

display(
    df[df['t_id'].isin(duplicated_t_ids)]
    .sort_values(by='t_id')
)

Comprobamos que un lote solo tiene un tipo de fruta y una sola marca.

In [ ]:
lote_check = df.groupby('lote').agg({
    'tipo': 'nunique',
    'marca': 'nunique'
}).reset_index()

tipo_check = lote_check[lote_check['tipo'] > 1]
marca_check = lote_check[lote_check['marca'] > 1]

if tipo_check.empty and marca_check.empty:
    print("Cada lote solo contiene un tipo de fruta o una marca.")
else:
    print("Norma incumplida:")
    if not tipo_check.empty:
        print("\nLotes con más de un tipo de fruta:")
        display(tipo_check)
    if not marca_check.empty:
        print("\nLotes con más de una marca:")
        display(marca_check)

Importamos SQLITE3 para futuras queries con los datos nuevos en la BBDD.

In [ ]:
import sqlite3

conn = sqlite3.connect("pjupiter.db")

In [ ]:
df_ventas.to_sql(
    name="ventas",
    con=conn,
    if_exists="append",
    index=False
)